## Green Holdings 

In [2]:
from _config import *

### 1. CIK Mapping

In [3]:
db = wrds.Connection(wrds_username = wrds_id)
cik_map = db.raw_sql('''
    select crsp_fundno::int, comp_cik::int as cik
    from crsp.crsp_cik_map 
    where crsp_fundno is not NULL
    '''
).sort_values(['cik','crsp_fundno']).reset_index(drop = True)

Loading library list...
Done


In [4]:
print(f'nrow: {cik_map.shape[0]:,}')
cik_map.head()

nrow: 65,908


,crsp_fundno,cik
0,7350,2110
1,7351,2110
2,7352,2110
3,7353,2110
4,7354,2110


In [5]:
db = wrds.Connection(wrds_username = wrds_id)
fund_hdr = db.raw_sql(''' 
    select crsp_fundno::int, crsp_portno::int, end_dt, fund_name
    from crsp.fund_hdr
    where crsp_portno is not NULL
    order by crsp_fundno
    ''',
    date_cols = ['end_dt']
)

Loading library list...
Done


In [6]:
print(f'nrow: {fund_hdr.shape[0]:,}')
fund_hdr.head()

nrow: 57,084


,crsp_fundno,crsp_portno,end_dt,fund_name
0,72,1005195,2000-05-31,Armada Funds: Bond Fund; I Shares
1,73,1005195,2000-05-31,Armada Funds: Bond Fund; A Shares
2,105,1022431,2024-12-31,American Pension Investors Trust: Yorktown Gro...
3,139,1003340,2000-08-31,"Atlas Assets, Inc.: Atlas Growth and Income Fu..."
4,184,1003387,2000-05-31,"Warburg Pincus Emerging Markets Fund, Inc.; In..."


In [ ]:
hdr = fund_hdr.merge(cik_map, on='crsp_fundno')
hdr = hdr.drop_duplicates(subset = ['cik','crsp_portno']).drop(columns = 'crsp_fundno').reset_index(drop = True)

hdr = hdr[['cik','crsp_portno','fund_name','end_dt']].sort_values(['cik','crsp_portno','end_dt']).reset_index(drop = True)

In [10]:
print(f'nrow: {hdr.shape[0]:,}')
hdr.head()

nrow: 21,797


,cik,crsp_portno,fund_name,end_dt
0,2110,1022184,Columbia Acorn Trust: Columbia Acorn Fund; Cla...,2024-12-31
1,2110,1023300,Columbia Acorn Trust: Columbia Acorn Internati...,2024-12-31
2,2110,1024481,Columbia Acorn Trust: Columbia Acorn USA Fund;...,2023-03-31
3,2110,1025279,Columbia Acorn Trust: Columbia Acorn Internati...,2024-12-31
4,2110,1026669,Columbia Acorn Trust: Columbia Thermostat Fund...,2024-12-31


### 2. MFRR

In [ ]:
mfrr = pd.read_parquet('output/mfrr/comp_excl_policy.parquet').sort_values(['cik','filing_date']).reset_index(drop = True)

# 0 -> 1
mfrr['flag_excl_l1'] = mfrr.groupby('cik')['flag_excl'].shift(1).fillna(0).astype(int)

mask = (mfrr['flag_excl'] == 1) & (mfrr['flag_excl_l1'] == 0)
mfrr['flag_treated'] = np.where(mask, 1, 0)

mfrr['period'] = mfrr.groupby('cik')['flag_treated'].cumsum()
mfrr['ldate'] = mfrr.groupby('cik')['filing_date'].shift(-1)

sub = mfrr.query('flag_treated == 1').reset_index(drop = True)
sub['ldate'] = sub['ldate'].fillna(pd.to_datetime(datetime.date.today()))

sub = sub.groupby(['cik','period'], as_index = False).agg(fdate = ('filing_date','min'), ldate = ('ldate','max')).sort_values(['cik','fdate']).reset_index(drop = True)

In [ ]:
hdr = hdr.merge(sub, on='cik')
hdr = hdr.query('fdate <= end_dt <= ldate').reset_index(drop = True)

hdr = hdr.groupby(['crsp_portno','period'], as_index = False).agg(fund_name = ('fund_name','first'), fdate = ('fdate','min'), ldate = ('ldate','max'))

In [ ]:
print(f'nrow: {hdr.shape[0]:,}')
hdr.head()

nrow: 1,619


,crsp_portno,period,fund_name,fdate,ldate
0,1000746,1,VanEck Funds: International Investors Gold Fun...,2021-10-13,2025-03-09
1,1001453,1,Russell Investment Company: Tax-Managed US Lar...,2018-09-28,2025-03-09
2,1001786,1,Russell Investment Company: Tax-Managed US Mid...,2018-09-28,2025-03-09
3,1005040,1,Russell Investment Company: Investment Grade B...,2018-09-28,2025-03-09
4,1005242,1,Russell Investment Company: Tax Exempt Bond Fu...,2018-09-28,2025-03-09


### 3. Holdings

In [33]:
hld = pd.read_parquet('data/mutual_funds/raw_crsp_mf_holdings.parquet')
hld['date'] = hld['report_dt'] + MonthEnd(0)
hld = hld[hld['sharesheld'] > 0].drop(columns=['report_dt','eff_dt']).reset_index(drop = True)

In [34]:
# Get stock price and compute position value
db = wrds.Connection(wrds_username = wrds_id)
crspm = db.raw_sql('''
    select a.permno::int, a.date, a.prc, a.shrout
    from crsp.msf as a 
    left join crsp.msenames as b 
        on a.permno=b.permno
        and b.namedt<=a.date
        and a.date<=b.nameendt
    where a.date between '01/01/2000' and '12/31/2024'
        and b.shrcd in (10, 11)
        and b.exchcd between 1 and 3
    ''', 
    date_cols = ['date']
).assign(
    date = lambda x: x['date'] + MonthEnd(0)
)

crspm['prc'] = crspm['prc'].mask(crspm['prc'] == -99999, np.nan)
crspm['prc'] = crspm['prc'].abs()
crspm['shrout'] = crspm['shrout'] / 1e3
crspm['me'] = crspm['prc'] * crspm['shrout']

Loading library list...
Done


In [35]:
hld = hld.merge(crspm, on = ['permno','date'], how = 'left')
hld['valueheld'] = hld['sharesheld'] * hld['prc'] / 1e6     # Position value in millions
hld['valueheld'] = hld['valueheld'].fillna(hld['market_val'] / 1e6)

hld = hld[['crsp_portno','permno','date','valueheld','me']].sort_values(['crsp_portno','permno','date']).reset_index(drop = True)

In [64]:
hld.query("permno == 10001 & date.dt.year == 2011 & date.dt.month == 12")

,crsp_portno,permno,date,valueheld,me
37460520,1022967,10001,2011-12-31,0.228389,93.11868
44871472,1023792,10001,2011-12-31,0.224974,93.11868
53391932,1024865,10001,2011-12-31,0.181578,93.11868
53718492,1024903,10001,2011-12-31,0.022258,93.11868
54355518,1024904,10001,2011-12-31,0.110991,93.11868
62130917,1025514,10001,2011-12-31,0.003426,93.11868
71242883,1025980,10001,2011-12-31,0.028276,93.11868
72293720,1026038,10001,2011-12-31,0.004397,93.11868
74376402,1026205,10001,2011-12-31,0.08565,93.11868
86881041,1027484,10001,2011-12-31,0.203687,93.11868


In [36]:
port = hld[['crsp_portno','date']].drop_duplicates().sort_values(['crsp_portno','date']).reset_index(drop = True)
port = port.merge(hdr, on = 'crsp_portno').query('date >= fdate & date <= ldate').reset_index(drop = True)

port = port[['crsp_portno','date']]
port['flag_treated'] = 1

In [37]:
port

,crsp_portno,date,flag_treated
0,1000746,2021-12-31,1
1,1000746,2022-03-31,1
2,1000746,2022-06-30,1
3,1000746,2022-09-30,1
4,1000746,2022-12-31,1
...,...,...,...
17757,1054263,2023-11-30,1
17758,1054263,2023-12-31,1
17759,1054342,2023-12-31,1
17760,1054344,2023-12-31,1


In [38]:
hld = hld.merge(port, on = ['crsp_portno','date'], how = 'left')
hld['flag_treated'] = hld['flag_treated'].fillna(0).astype(int)

hld['grn_hld'] = hld['valueheld'] * hld['flag_treated'] 

In [40]:
grn = hld.groupby(['permno','date'], as_index = False).agg(grn_hld = ('grn_hld','sum'), mf_hld = ('valueheld','sum'), me = ('me','first'), n_funds = ('crsp_portno','size'), n_treated = ('flag_treated','sum'))

grn['treated_me'] = grn['grn_hld'] / grn['me']
grn['n_treated_mf'] = grn['n_treated'] / grn['n_funds']
grn['mf_me'] = grn['mf_hld'] / grn['me']
grn['treated_mf'] = grn['grn_hld'] / grn['mf_hld']

In [41]:
inst = pd.read_parquet('../FDATA/fs_processed/hld_v0.parquet', columns = ['date','inst_id','permno','adj_mv']).assign(eoq = lambda x: pd.to_datetime(x['date'], format = r'%Y%m%d'), valueheld = lambda x: x['adj_mv'] / 1e6).drop(columns = 'adj_mv').query('inst_id != 0').reset_index(drop = True)
inst = inst.groupby(['permno','eoq'], as_index = False).agg(inst_hld = ('valueheld','sum'))

In [ ]:
grn['eoq'] = grn['date'] + QuarterEnd(0)

mask = (grn['date'] != grn['eoq'])
grn.loc[mask, 'eoq'] = grn.loc[mask, 'date'] + QuarterEnd(-1)

# Scale by total institutional holdings
grn = grn.merge(inst, on = ['permno','eoq'], how = 'left')

mask = (grn['inst_hld'] > grn['me'])
grn.loc[mask, 'inst_hld'] = grn.loc[mask, 'me']

# Construct variables
grn['mf_inst'] = grn['mf_hld'] / grn['inst_hld']

grn.loc[grn['mf_inst'] > 1, 'flag_non_overlapping'] = 1

grn['flag_non_overlapping'] = grn['flag_non_overlapping'].fillna(0).astype(int)

In [49]:
hld.query('permno == 10001 & date.dt.year == 2007')

,crsp_portno,permno,date,valueheld,me,flag_treated,grn_hld
1384540,1000379,10001,2007-03-31,0.024744,43.43894,0,0.0
1384541,1000379,10001,2007-06-30,0.033,42.885,0,0.0
3913905,1001028,10001,2007-01-31,0.054395,33.609506,0,0.0
3913906,1001028,10001,2007-02-28,0.066537,42.41391,0,0.0
3913907,1001028,10001,2007-03-31,0.06785,43.43894,0,0.0
3913908,1001028,10001,2007-04-30,0.068037,43.55902,0,0.0
3913909,1001028,10001,2007-07-31,0.066349,40.45485,0,0.0
3913910,1001028,10001,2007-10-31,0.059541,38.11425,0,0.0
4236151,1001116,10001,2007-05-31,0.056962,45.01497,0,0.0
7023242,1001706,10001,2007-02-28,0.039732,42.41391,0,0.0


In [46]:
grn

,permno,date,grn_hld,mf_hld,me,n_funds,n_treated,treated_me,n_treated_mf,mf_me,treated_mf,eoq,inst_hld,mf_inst,flag_non_overlapping
0,10001,2002-09-30,0.0,0.051625,22.54,1,0,0.0,0.000000,0.00229,0.0,2002-09-30,0.387039,0.133384,0
1,10001,2002-11-30,0.0,0.002512,21.682875,2,0,0.0,0.000000,0.000116,0.0,2002-09-30,0.387039,0.006492,0
2,10001,2002-12-31,0.0,0.033815,19.046441,4,0,0.0,0.000000,0.001775,0.0,2002-12-31,0.302369,0.111832,0
3,10001,2003-03-31,0.0,0.050016,19.83645,3,0,0.0,0.000000,0.002521,0.0,2003-03-31,0.320023,0.156288,0
4,10001,2003-05-31,0.0,0.001688,21.89336,1,0,0.0,0.000000,0.000077,0.0,2003-03-31,0.320023,0.005275,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455066,93436,2023-08-31,1592.430962,129646.365656,819144.33978,836,72,0.001944,0.086124,0.15827,0.012283,2023-06-30,NaN,<NA>,0
1455067,93436,2023-09-30,1641.62536,158120.395247,795449.38,1145,96,0.002064,0.083843,0.198781,0.010382,2023-09-30,NaN,<NA>,0
1455068,93436,2023-10-31,1667.615491,103137.366885,638454.49364,844,83,0.002612,0.098341,0.161542,0.016169,2023-09-30,NaN,<NA>,0
1455069,93436,2023-11-30,1550.643829,123281.914912,763195.35368,821,77,0.002032,0.093788,0.161534,0.012578,2023-09-30,NaN,<NA>,0


In [ ]:
# grn.to_parquet('output/mfrr/green_holdings_0310.parquet', index = False)

print(f'nrow: {grn.shape[0]:,}')
grn.head()

nrow: 1,455,071


,permno,date,grn_hld,mf_hld,me,n_funds,n_treated,treated_me,n_treated_mf,mf_me,treated_mf,eoq,inst_hld,mf_inst,flag_non_overlapping
0,10001,2002-09-30,0.0,0.051625,22.54,1,0,0.0,0.0,0.00229,0.0,2002-09-30,0.387039,0.133384,0
1,10001,2002-11-30,0.0,0.002512,21.682875,2,0,0.0,0.0,0.000116,0.0,2002-09-30,0.387039,0.006492,0
2,10001,2002-12-31,0.0,0.033815,19.046441,4,0,0.0,0.0,0.001775,0.0,2002-12-31,0.302369,0.111832,0
3,10001,2003-03-31,0.0,0.050016,19.83645,3,0,0.0,0.0,0.002521,0.0,2003-03-31,0.320023,0.156288,0
4,10001,2003-05-31,0.0,0.001688,21.89336,1,0,0.0,0.0,0.000077,0.0,2003-03-31,0.320023,0.005275,0


## END